In [41]:
from qutip import *
from numpy import savetxt, real, pi

In [17]:
s0 = Qobj([[1, 0], [0, 0]])
s1 = Qobj([[0, 0], [0, 1]])
I = qeye(2)
rho111 = tensor(s1,s1,s1,I,I,I)
z1 = tensor(sigmax(),I,I,I,I,I)
z2 = tensor(I,sigmax(),I,I,I,I)
z3 = tensor(I,I,sigmax(),I,I,I)
HP = -0.2*(z1*z2 + z1*z3 + z2*z3)

In [40]:
somemat = real((HP*rho111).full())
savetxt('doink.txt', somemat)

In [ ]:
def lindbladME(cp, cs, rho, H):
    return 2j*pi*(rho*H - H*rho) + cp*(x)

In [ ]:
 inline void basic_funcs::lindbladME(float cp, float cs, MatrixXcd& rho, MatrixXcd& H, MatrixXcd& output) {
    output = 2.0*IM*PI*(rho*H - H*rho) 
             + cp*(X1*rho*X1 - 0.5*(X1*rho + rho*X1)) + cs*(a1*rho*a1d - 0.5*(a1d*a1*rho + rho*a1d*a1))
             + cp*(X2*rho*X2 - 0.5*(X2*rho + rho*X2)) + cs*(a2*rho*a2d - 0.5*(a2d*a2*rho + rho*a2d*a2))
             + cp*(X3*rho*X3 - 0.5*(X3*rho + rho*X3)) + cs*(a3*rho*a3d - 0.5*(a3d*a3*rho + rho*a3d*a3));
    return;
}

inline void basic_funcs::lindbladRK4(float col1, float col2, float step, MatrixXcd& rho, MatrixXcd& H, MatrixXcd& output) {
    MatrixXcd t1, t2, t3, k1, k2, k3, k4;
    lindbladME(col1, col2, rho, H, k1);
    t1 = rho + 0.5*step*k1; lindbladME(col1, col2, t1, H, k2);
    t2 = rho + 0.5*step*k2; lindbladME(col1, col2, t2, H, k3);
    t3 = rho + step*k3; lindbladME(col1, col2, t3, H, k4);
    output = rho + step*(k1 + 2*k2 + 2*k3 + k4)/6.0;
    return;
}